In [10]:
import numpy as np
import pickle 
import pandas as pd
import matplotlib.pyplot as plt
import sys 
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
WORK_PATH = "/Users/welcome/Google Drive (cuong.tranus@gmail.com)/research/internship/fraud-detection"

In [11]:
pd00 = pd.read_json(WORK_PATH + '/data/transactions.txt', lines=True)
pd00.head(3)

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,,0.0,,,,False,,,False,False
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,,0.0,,,,True,,,False,False
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,,0.0,,,,False,,,False,False


In [12]:
feat_dict = pickle.load(open(WORK_PATH + "/results/features_category.pkl" ,"rb"))
remove_cols = feat_dict['remove_cols']
print(remove_cols)

['acqCountry', 'merchantCountryCode', 'echoBuffer', 'merchantCity', 'merchantState', 'merchantZip', 'posOnPremises', 'recurringAuthInd', 'expirationDateKeyInMatch']


In [13]:
used_cols = [col for col in pd00.columns.tolist() if col not in remove_cols]
pd00 = pd00[used_cols]

In [14]:
pd00['date'] = pd00['transactionDateTime'].apply(lambda x: x.split("T")[0]).astype(str)
pd00[['accountNumber','date']].head(1)

,accountNumber,date
0,737265056,2016-08-13


In [23]:
np.mean( np.random.rand(1000))

0.5070227041653669

In [38]:
def get_tvt_code(date, seed):
    """
    Split each records(row) into train/val/test partition
    Return train/val/test code for each row
    
    """
    if date >='2016-10-01':
        return 'test'
    else:
        random_number = np.random.rand()
        if random_number >=0.75:
            return 'val_{}'.format(seed)
        else:
            return 'train_{}'.format(seed)

# Fix test set (out-of-sample), but for trainval(in-of-sample case) create like(not exactly) five folds.

for seed in range(5):
    pd00['tvt_code_{}'.format(seed)] = pd00['date'].apply(lambda x: get_tvt_code(x, seed))


In [39]:
# Sanity check
for seed in range(5):
    print(pd00['tvt_code_{}'.format(seed)].value_counts())
    print("#####################")

train_0    434430
test       206700
val_0      145233
Name: tvt_code_0, dtype: int64
#####################
train_1    434307
test       206700
val_1      145356
Name: tvt_code_1, dtype: int64
#####################
train_2    434547
test       206700
val_2      145116
Name: tvt_code_2, dtype: int64
#####################
train_3    434995
test       206700
val_3      144668
Name: tvt_code_3, dtype: int64
#####################
train_4    435022
test       206700
val_4      144641
Name: tvt_code_4, dtype: int64
#####################


In [40]:
pd00[['accountNumber','date','tvt_code_0','tvt_code_1','tvt_code_2','tvt_code_3','tvt_code_4']].head(3)

,accountNumber,date,tvt_code_0,tvt_code_1,tvt_code_2,tvt_code_3,tvt_code_4
0,737265056,2016-08-13,train_0,train_1,train_2,train_3,val_4
1,737265056,2016-10-11,test,test,test,test,test
2,737265056,2016-11-08,test,test,test,test,test


In [41]:
pd00['isFraud'] = pd00['isFraud'].astype(int)

In [43]:
pd00.to_csv(WORK_PATH + '/data/processed_data.csv', index = False)